In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
from torchvision.utils import save_image

augmentation = transforms.Compose([
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.3),
    transforms.Resize((128, 128)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

In [2]:
# set the path
original_data_path = './jaffe/'  # folder of original images
augmented_data_path = './jaffe_augmented/' # folder of augmented images
os.makedirs(augmented_data_path, exist_ok=True)

# save the augumented images
img_extensions = ['.tiff', '.jpg', '.png']
for img_file in os.listdir(original_data_path):
    if any(img_file.endswith(ext) for ext in img_extensions):
        img_path = os.path.join(original_data_path, img_file)
        img = Image.open(img_path).convert("L")  # convert to grayscale
        base_name = os.path.splitext(img_file)[0]

        for i in range(21):  # original image + 20 variations
            transformed_img = augmentation(img)
            save_path = os.path.join(augmented_data_path, f"{base_name}_{i}.jpg")
            save_image(transformed_img, save_path)

In [3]:
# load the augumented images and label them
img_data_list = []
labels = []
names = ['AN', 'DI', 'FE', 'HA', 'NE', 'SA', 'SU']

# emotion name and label correspondence table
emotion_map = {
    'AN': 0,  # ANGRY
    'DI': 1,  # DISGUST
    'FE': 2,  # FEAR
    'HA': 3,  # HAPPY
    'NE': 4,  # NEUTRAL
    'SA': 5,  # SAD
    'SU': 6   # SURPRISE
}

# load the augumented images
for img_file in os.listdir(augmented_data_path):
    # check if the image is a jpg
    if img_file.endswith('.jpg'):
        img_path = os.path.join(augmented_data_path, img_file)
        img = cv2.imread(img_path)
        # convert to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # resize the image
        resized_img = cv2.resize(gray_img, (128, 128))
        # append the image to the list
        img_data_list.append(resized_img)

        # get the label from the file name
        for code, label in emotion_map.items():
            if code in img_file:
                labels.append(label)
                break

            resized_img = cv2.resize(gray_img, (128, 128))
            img_data_list.append(resized_img)

            # extract the label from the file name
            for code, label in emotion_map.items():
                if code in img_file:
                    labels.append(label)
                    break

# convert to numpy array and normalize
img_data = np.array(img_data_list, dtype='float32') / 255.0
img_data = np.expand_dims(img_data, axis=1)  # (N, 1, 128, 128)
labels = np.array(labels, dtype='int64')

# print the total number of images and the distribution of labels
print(f"Total images loaded: {img_data.shape[0]}")
print("Label distribution:")
unique, counts = np.unique(labels, return_counts=True)
for label, count in zip(unique, counts):
    print(f"{label}: {count} images")

Total images loaded: 17934
Label distribution:
0: 630 images
1: 1218 images
2: 2016 images
3: 2604 images
4: 3150 images
5: 3906 images
6: 4410 images
